<a href="https://colab.research.google.com/github/asheta66/Machine-Learning-2020/blob/main/NLP%20Programming/chapter_30/4_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Acquireing datasets
!git clone https://github.com/asheta66/Machine-Learning-2020
# Create Model via another notebook
%run Machine-Learning-2020/NLP\ Programming/chapter_30/3_train_model.ipynb

Cloning into 'Machine-Learning-2020'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 3107 (delta 91), reused 3 (delta 0), pack-reused 2977
Receiving objects: 100% (3107/3107), 37.29 MiB | 5.75 MiB/s, done.
Resolving deltas: 100% (735/735), done.
fatal: destination path 'Machine-Learning-2020' already exists and is not an empty directory.
fatal: destination path 'Machine-Learning-2020' already exists and is not an empty directory.
fatal: destination path 'Machine-Learning-2020' already exists and is not an empty directory.
/content/Machine-Learning-2020/Datasets/NLP
Saved: english-german.pkl
[go] => [geh]
[hi] => [hallo]
[hi] => [gru gott]
[run] => [lauf]
[run] => [lauf]
[wow] => [potzdonner]
[wow] => [donnerwetter]
[fire] => [feuer]
[help] => [hilfe]
[help] => [zu hulf]
[stop] => [stopp]
[stop] => [anhalten]
[wait] => [warte]
[wait] => [warte]
[begin] => [fang an]
[go on] => [mac

In [2]:
from pickle import load
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i][:2]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append(raw_target.split())
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# load datasets
dataset = load_clean_sentences('english-german-both.pkl')
train = load_clean_sentences('english-german-train.pkl')
test = load_clean_sentences('english-german-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
# load model
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, testX, test)

train
src=[sie hat eine erkaltung], target=[she has a cold], predicted=[she has a cold]
src=[sie sind leer], target=[theyre empty], predicted=[theyre empty]
src=[ich habe ein flugzeug gesehen], target=[i saw a plane], predicted=[i saw a plane]
src=[verkrumele dich], target=[get lost], predicted=[go away]
src=[ich laufe gern], target=[i like running], predicted=[i like to]
src=[das ist ihre aufgabe], target=[its their job], predicted=[its their job]
src=[ich koche gerade], target=[im baking], predicted=[im eating]
src=[niemand schlief], target=[nobody slept], predicted=[nobody slept]
src=[ich bin dreizehn], target=[im thirteen], predicted=[im thirteen]
src=[das geht auf mich], target=[it is on me], predicted=[its for me me]


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.077043
BLEU-2: 0.263357
BLEU-3: 0.430594
BLEU-4: 0.486911
test
src=[tom kann gut singen], target=[tom sings well], predicted=[tom may win]
src=[es ist ernst gemeint von mir], target=[i mean it], predicted=[its not me]
src=[gute arbeit tom], target=[good work tom], predicted=[where this tom]
src=[ich sah die nachrichten], target=[i saw the news], predicted=[i saw you]
src=[iss bitte], target=[please eat], predicted=[please eat]
src=[tom stellte eine falle], target=[tom set a trap], predicted=[tom had mary]
src=[ihr seid damlich], target=[youre silly], predicted=[youre silly]
src=[ich mag pferde], target=[i like horses], predicted=[i like confused]
src=[keine panik], target=[dont panic], predicted=[stop down]
src=[geht spazieren], target=[take a walk], predicted=[take a a]
BLEU-1: 0.080568
BLEU-2: 0.268402
BLEU-3: 0.434345
BLEU-4: 0.489888
